# Case

Naive sentiment analysis

This is a simple project using Naive Bayes Classifier and Scikit-learn to create a Google Play store reviews classifier (Sentiment Analysis) in Python. You will categorize user reviews as good or bad. The Naive Bayes classification technique is a simple and powerful classification task in machine learning. The Naive Bayes classification technique is a simple and powerful classification task in machine learning. In this dataset, we use the 23 most popular mobile apps, and only two columns.

Step 1:

We have three columns: package name, review and polarity (0 = bad, 1 = good) Preprocess the data by eliminating the package name column and putting all reviews in lower case.

Step 2:

Separate target from feature, and split your data.

Step 3:

Vectorize your features and use Naive Bayes to classify the reviews as good or bad. We will not focus on hypertuning our model this time. This was an introduction project to sentiment analysis using Naive Bayes.

Step 4:

Use the app.py to create your pipeline.

Save your naive bayes classification model in the 'models' folder.

Insall de libraries

In [424]:
!pip install pandas
!pip install numpy
!pip install sklearn
!pip install nltk
!pip install matplotlib
%pip install pickle

18576.96s - pydevd: Sending message related to process being replaced timed-out after 5 seconds



[notice] A new release of pip available: 22.2 -> 22.2.2
[notice] To update, run: pip install --upgrade pip


18584.50s - pydevd: Sending message related to process being replaced timed-out after 5 seconds



[notice] A new release of pip available: 22.2 -> 22.2.2
[notice] To update, run: pip install --upgrade pip


18592.33s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


...


[notice] A new release of pip available: 22.2 -> 22.2.2
[notice] To update, run: pip install --upgrade pip


18607.61s - pydevd: Sending message related to process being replaced timed-out after 5 seconds



[notice] A new release of pip available: 22.2 -> 22.2.2
[notice] To update, run: pip install --upgrade pip


18615.11s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


ERROR: Could not find a version that satisfies the requirement pickle (from versions: none)
ERROR: No matching distribution found for pickle

[notice] A new release of pip available: 22.2 -> 22.2.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


Imports

In [425]:
import pandas as pd
import numpy as np
import nltk #text processing
import re
import unicodedata
import pickle
import matplotlib.pyplot as plt
%matplotlib inline
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report
from sklearn.naive_bayes import GaussianNB



In [426]:
#load the words to use as filter
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/gitpod/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Load Data of Reviews

In [427]:
url='https://raw.githubusercontent.com/4GeeksAcademy/naive-bayes-project-tutorial/main/playstore_reviews_dataset.csv'
df = pd.read_csv(url)

Explore Dataset

In [428]:
df.head()


,package_name,review,polarity
0,com.facebook.katana,privacy at least put some option appear offline. i mean for some people like me it's a big pressure to be seen online like you need to response on every message or else you be called seenzone only. if only i wanna do on facebook is to read on my newsfeed and just wanna response on message i want to. pls reconsidered my review. i tried to turn off chat but still can see me as online.,0
1,com.facebook.katana,"messenger issues ever since the last update, initial received messages don't get pushed to the messenger app and you don't get notification in the facebook app or messenger app. you open the facebook app and happen to see you have a message. you have to click the icon and it opens messenger. subsequent messages go through messenger app, unless you close the chat head... then you start over with no notification and having to go through the facebook app.",0
2,com.facebook.katana,profile any time my wife or anybody has more than one post and i view them it would take me to there profile so that i can view them all at once. now when i try to view them it tells me that the page that i requested is not available. i've restarted my phone i even cleard the cache and i've uninstalled and reinstalled the app and it is still doing it. please fix it thank you,0
3,com.facebook.katana,the new features suck for those of us who don't have a working back button can you guys make the videos able to be slid to the left to exit the video. as i have to force close facebook to exit,0
4,com.facebook.katana,"forced reload on uploading pic on replying comment last night i tried to reply a comment by uploading a photo from my phone. when i press on the button to select photos the app automatically goes back to the main page. on other occasions, i could enter to my gallery to select my image but as soon as i selected an image and press done, the program, again, forced reload and suddenly go back to the main page. please fix this and i will change my rating. thank you.",0


In [429]:
df['polarity'].value_counts()

0    584
1    307
Name: polarity, dtype: int64

In [430]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   package_name  891 non-null    object
 1   review        891 non-null    object
 2   polarity      891 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 21.0+ KB


Package name, review and polarity (0 = bad, 1 = good)

In [431]:
df.sample(10)

,package_name,review,polarity
40,com.twitter.android,⭐⭐⭐ when will this app be created to automatically start at the top with new posts instead of having to scroll through days of old tweets to get to the top! the only social app that does that! such a headache!,0
678,com.hamrokeyboard,i found this app very fruitful.. and m using it but can you please tell me how to write ŕ¤ŕ¤§ŕ¤ž ŕ¤ ŕ¤ŕľŕ¤ˇŕ¤°,1
18,com.facebook.katana,"can't connect right now when trying to open links in the facebook browser, i receive a can't connect right now error message. other items on my news feed continue to load.",0
586,com.evernote,"widget sabotaged this app functions ok as a note-taker. i access it primarily through its widget. the old widget was great, giving a peek at a few recent notes. then that widget was killed off, and replaced with a widget that shows only the titles of 2 recent notes, with maybe just part of the title of a third note, and no content at all. it shows poor judgment for a company to downgrade its product for no apparent reason. also, there's no undo button.",1
498,com.Slack,tablet support any chance of getting tablet support anytime soon?,0
501,com.Slack,slick chat app! reactions are my favorite part of using slack. i love adding my own!,1
491,com.Slack,good this is a very good app so far. great features and overall performance.,1
36,com.facebook.katana,update uninstalls! everytime i try to update it uninstalls the app! what i want to know is if this is using my data each time i update and then have to reinstall?! driving me crazy. please fix.,0
847,com.hamropatro,good ŕ¤ŕ¤ŕ¤śŕ¤ž ŕ¤°ŕ¤žŕ¤žŕľŕ¤°ŕľ,1
566,jabanaki.todo.todoly,doesn't log in checked my credentials multiple times. will not authenticate.,0



Text Processing - Clean

Step 1 - change all to lower case


In [432]:
#Preprocess the data by eliminating the package name column and putting all reviews in lower case.

df.drop(['package_name'],axis=1, inplace=True)
#chante all to lower case
df['review'] = df['review'].str.lower()




Step 2 - Remove all words that not have meaning to the search - stop words


In [433]:
#see, the word in the reviews, create a list of word and frecuency
df['review'].str.split(expand = True).stack().value_counts()[:60] #(show top 60)

the       1293
to        1159
i         1084
and        853
it         778
a          619
is         555
my         442
this       421
for        410
of         390
but        348
in         340
on         325
not        318
app        317
you        315
that       260
have       252
with       220
so         203
be         194
no         186
when       182
if         180
or         169
all        163
can        158
as         148
me         145
are        139
it's       138
can't      137
update     136
new        136
like       135
fix        133
please     131
good       128
game       121
one        119
use        118
just       117
get        112
its        111
very       109
more       107
was        104
there      104
don't      103
now        102
will       102
even       101
from       101
at         100
up          99
great       99
time        93
your        90
do          89
dtype: int64

In [434]:
#remove the stopword
stop = stopwords.words('english')

def remove_stopwords(text):
  if text is not None:
    #list of the word in the text
    words = text.strip().split()
    words_filtered = []
    for word in words:
      if word not in stop:
        words_filtered.append(word)
    result = " ".join(words_filtered) #join the word in a text with space separation
  else:
      result = None
  return result




Eliminamos los caracteres desconocidos


In [435]:
# elimina espacio libre al principio y al final
df['review'] = df['review'].str.strip()
df['review'] = df['review'].str.lower()


In [436]:
df['review']


0                                                                                      privacy at least put some option appear offline. i mean for some people like me it's a big pressure to be seen online like you need to response on every message or else you be called seenzone only. if only i wanna do on facebook is to read on my newsfeed and just wanna response on message i want to. pls reconsidered my review. i tried to turn off chat but still can see me as online.
1               messenger issues ever since the last update, initial received messages don't get pushed to the messenger app and you don't get notification in the facebook app or messenger app. you open the facebook app and happen to see you have a message. you have to click the icon and it opens messenger. subsequent messages go through messenger app, unless you close the chat head... then you start over with no notification and having to go through the facebook app.
2                                                     

In [437]:
def normalize_str(text_string):
    if text_string is not None:
        result=unicodedata.normalize('NFD',text_string).encode('ascii','ignore').decode()
    else:
        result=None 
    return result

In [438]:
df['review']=df['review'].apply(normalize_str)
df['review']=df['review'].str.replace('!','')
df['review']=df['review'].str.replace(',','')
df['review']=df['review'].str.replace('&','')
df['review']=df['review'].str.normalize('NFKC')
df['review']=df['review'].str.replace(r'([a-zA-Z])\1{2,}',r'\1',regex=True)

In [439]:
df.sample(10)

,review,polarity
453,very nice app best feature is all friends get in touch with it.. runs well during roaming and with low signal.. i hope to see some improvement as in group chatting if we have tag feature or topic hashtag feature within the group so that we can refer to a particular topic in discussion just like in twitter etc.. but within the group not public ..would be great.,1
125,memory game the levels are so hard and less moves . l do no how to pass 95 levels but levels are so bour. chico bar is very hard to finish. please any of you send me life,1
305,after ur latest update i and my friend made a video call after ur latest update video of my friend z vry clr bt voice cnt b heard of my friend so plz plz resolve dis problem plz plz its a kind request,0
19,error i cannot edit my post with pictures. the moment i do it will not change after save and it appears like when ur uploading but not yet done post in news feed and still there until i did reinstall the app. this still happens even.,0
361,it's great when it works but right now it won't download any mms. it just says they're there and won't download them and then loses the message when i switch to a different app to try to download it instead.,1
47,came here to rate this app to give 5 stars for good optimization for android this is one of the only apps that i open and close and it actually clears fully from ram and does not use up cpu power unlike facebook and its messenger using up ram and cpu.,0
795,trust worthy as always have been a long time user of firefox in all my desktops and laptops. now using it on my android phones as well. not settling for any other second rate browser,1
879,not the same with my hand on my heart. i can honestly say that the first angry birds was great. the latter is a horrible mess of horrible games cashing in on the original success. unfortunately this hame has suffered the same fate... it is no longer the brilliant game it once was... it's an updated version with none of the original stuff that once made it great...,1
560,great simple recommend love this app. looking forward to more new features.,1
496,everyone loves slack. count me among them,1


In [440]:
df.iloc[675,]

review      loved it rzrl app rrlrrl rrsrzrlrrl r rrsrzrlrrl r rr rsrrz rzrrzrz rrlrsrl rrlrrz rlrsrl rr rzrrl rrsrzrlrrl rsrlrrl rlrzrzrl rlrrlr inserting symbols inserting pictures from the galary wedding 123 etc. and phonetic symbols so the english learners could be helped more. rrsrz rlrrl rrlrrz rrz rsrzrrzrz rrlrsrlrz rsrrz rzrl  rrtrlrrz rsrlrrzrlr rlrrzrsrrlrrzr rrsrzrlr rlrrl r rl
polarity                                                                                                                                                                                                                                                                                                                                                                                               1
Name: 675, dtype: object

In [441]:
df_interim = df.copy()

In [442]:
df_interim['review']=df_interim['review'].str.lower()

In [443]:
df_interim['review'] = df_interim['review'].apply(remove_stopwords)


In [444]:
df_interim['review'].str.split(expand=True).stack().value_counts()[:60] 

app         351
update      150
game        140
please      138
like        137
can't       137
new         137
fix         135
good        134
use         123
one         120
great       115
get         112
even        102
time         98
browser      85
really       84
back         81
still        80
would        80
love         77
make         74
5            71
work         71
also         69
see          69
using        68
phone        68
i'm          67
open         66
it.          65
want         64
google       64
much         62
every        61
version      60
option       59
u            58
way          57
app.         56
go           56
android      56
used         55
best         54
download     53
better       53
play         52
.            51
give         50
send         49
people       48
able         48
message      47
since        47
many         46
works        45
apps         45
ads          45
keep         45
old          44
dtype: int64

In [445]:
def word_only_letters(word):
    for c in word:
        cat = unicodedata.category(c)
        if cat not in ('Ll','Lu'):  #only letter upper y lower
            return False
    return True

# clean only letter
def text_only_letters(text):
    if text is not None:
        #list of the word in the text
        words = text.strip().split()
        words_filtered = []
        for word in words:
            if word_only_letters(word):
                words_filtered.append(word)
            result = " ".join(words_filtered) #join the word in a text with space separation
    else:
        result = None
    return result

In [446]:
df_interim['review'] = df_interim['review'].apply(text_only_letters)

In [447]:
#remove multi letter looove iiiitttt, or repeat secuence
def replace_multiple_letters(message):
  if message is not None:
    result = re.sub(r'(.+?)\1+', r'\1', message)
    #result = re.sub(r"([a-zA-Z])＼1{2,}", r"＼1", message)
  else:
    result = None
  return result


In [448]:
df_interim['review'] = df_interim['review'].apply(replace_multiple_letters)

In [449]:
df_interim['review']

0                                                                           privacy least put option apear mean people like big presure sen online like ned response every mesage else caled senzone wana facebok read newsfed wana response mesage want pls reconsidered tried turn chat stil se
1                                           mesenger isues ever since last update initial received mesages get pushed mesenger ap get notification facebok ap mesenger open facebok ap hapen se click icon opens subsequent mesages go mesenger ap unles close chat start notification go facebok
2                                                                                                                   profile time wife anybody one post view would take profile view try view tels page requested restarted phone even cleard cache uninstaled reinstaled ap stil please fix thank
3                                                                                                                                 

In [450]:
#copy to df
df = df_interim.copy()




Setp 3 - Model


In [451]:
# Separate target and predictor
X = df['review']
y = df['polarity']


In [452]:
vectorizer = CountVectorizer(stop_words='english')
X = vectorizer.fit_transform(X)
vectorizer.get_feature_names_out()

array(['abesta', 'abilities', 'ability', ..., 'zespole', 'zom', 'zoming'],
      dtype=object)

In [453]:
X = X.toarray()

In [454]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=25)

In [455]:
#Vectorizer change matriz with 0-1
#vec = CountVectorizer(stop_words = "english")
#X_train = vec.fit_transform(X_train).toarray()
#X_test = vec.transform(X_test).toarray()

In [456]:
print(X_train)


[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [457]:
print(X.shape)
print(X_test.shape)
print(X_train.shape)

(891, 3289)
(223, 3289)
(668, 3289)


In [458]:
clf = GaussianNB()
clf.fit(X_train, y_train)

GaussianNB()

In [459]:
y_train.value_counts()

0    438
1    230
Name: polarity, dtype: int64

In [460]:
y_train_pred = clf.predict(X_train)
y_test_pred = clf.predict(X_test)

In [461]:
target_names = ['bad', 'good']
print(classification_report(y_train, y_train_pred, target_names=target_names))

              precision    recall  f1-score   support

         bad       1.00      0.98      0.99       438
        good       0.96      1.00      0.98       230

    accuracy                           0.99       668
   macro avg       0.98      0.99      0.99       668
weighted avg       0.99      0.99      0.99       668



In [462]:
print(classification_report(y_test, y_test_pred, target_names=target_names))


              precision    recall  f1-score   support

         bad       0.80      0.83      0.81       146
        good       0.65      0.61      0.63        77

    accuracy                           0.75       223
   macro avg       0.73      0.72      0.72       223
weighted avg       0.75      0.75      0.75       223



No resultó muy bueno - overfit

# Naive Bayes

Naive Bayes is a probabilistic algorithm for classification, based on Bayes Theorem. This algorithm really makes you understand a big deal about classification and it is probably one of the most useful algorithms for simple Machine Learning use cases. It is simple enough to be completed by hand.

In [463]:
clf_multinomial = MultinomialNB()
clf_multinomial.fit(X_train, y_train)

MultinomialNB()

In [464]:
MultinomialNB()


MultinomialNB()

In [465]:
y_train_pred = clf_multinomial.predict(X_train)
y_test_pred = clf_multinomial.predict(X_test)


In [466]:
print(classification_report(y_train, y_train_pred, target_names=target_names))


              precision    recall  f1-score   support

         bad       0.95      0.97      0.96       438
        good       0.95      0.90      0.92       230

    accuracy                           0.95       668
   macro avg       0.95      0.94      0.94       668
weighted avg       0.95      0.95      0.95       668



In [467]:
print(classification_report(y_test, y_test_pred, target_names=target_names))


              precision    recall  f1-score   support

         bad       0.86      0.90      0.88       146
        good       0.79      0.71      0.75        77

    accuracy                           0.83       223
   macro avg       0.82      0.81      0.81       223
weighted avg       0.83      0.83      0.83       223



In [468]:
Z = vectorizer.transform(['I like this app'])

In [469]:
print(np.array(Z))
print(Z.shape)
print(Z)

  (0, 142)	1
  (0, 1602)	1
(1, 3289)
  (0, 142)	1
  (0, 1602)	1


In [470]:
# Modelo GaussianNB
print(clf.predict(Z.toarray()))



[1]


In [471]:
# Modelo Multinomial
print(clf_multinomial.predict(Z.toarray()))


[1]


In [472]:
pd.set_option('display.max_colwidth', None)
df.tail(10)

,review,polarity
881,game ruined felt like playing awhile imediately second awful mobile game ad poped second level loking store realized pay remove yep,0
882,ads way many ads even enjoy game amount ads every level,0
883,great game many ads almost worth,1
884,fun hard angry birds realy fun video game people least year olds people lot years older one really chalenge teach children control even though ba litle,1
885,many ads far adverts game know fre ned ads make profit neds,1
886,loved incredible awesome go power make new clash clans thing but beter,1
887,time legendary game birthday party levels short fuse levels pigs crash onto diferent chemicals players cringe much ads close ur conection play ads trouble,1
888,ads way heavy listen bad ads every round whether pas fail sometimes ads next round starts spend time ads game develop web aps honestly many people rely ads make apreciate however developers went far beyond frankly disrespectful,0
889,fun works perfectly ads anoying think especialy fre,1
890,everywhere se angry birds everywhere stop playing get head devs perfect,1


In [473]:
Z2 = vectorizer.transform(['I do not like this app'])

# Modelo GaussianNB
print(f"Gauss: {clf.predict(Z2.toarray())}")

# Modelo Multinomial
print(f"Multi: {clf_multinomial.predict(Z2.toarray())}")



Gauss: [1]
Multi: [1]


Guardar Modelo

In [474]:
pickle.dump(clf_multinomial, open('../models/multinomial.pkl', 'wb'))

Traer el Modelo

In [475]:
load_model = pickle.load(open('../models/multinomial.pkl', 'rb'))


In [476]:
Z2 = vectorizer.transform(['I do not like this app'])
# Modelo Multinomial loaded
print(f"Multi: {load_model.predict(Z2.toarray())}")



Multi: [1]


In [477]:
!pip install torch
!pip install tensorflow
!pip install -q transformers

from transformers import pipeline
sentiment_pipeline = pipeline("sentiment-analysis")

18637.60s - pydevd: Sending message related to process being replaced timed-out after 5 seconds



[notice] A new release of pip available: 22.2 -> 22.2.2
[notice] To update, run: pip install --upgrade pip


18645.24s - pydevd: Sending message related to process being replaced timed-out after 5 seconds



[notice] A new release of pip available: 22.2 -> 22.2.2
[notice] To update, run: pip install --upgrade pip


18653.17s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [478]:
data = ["I love you", "I hate you"]
sentiment_pipeline(data)

[{'label': 'POSITIVE', 'score': 0.9998656511306763},
 {'label': 'NEGATIVE', 'score': 0.9991129040718079}]

In [479]:
nb.predict(vec.transform(['i don\'t like','eat much ram','eat','like']))


array([0, 0, 0, 0])

In [480]:
nb.predict(vec.transform(['love','great solid love important feature','ram','like fast']))

array([1, 1, 0, 1])

In [481]:
df[df['polarity']==1]['review'].sample(10)

679                              hamro keyboard god ap doubt however ading features definitely make best folowing things option caps lock key enabled made available leters available along also les options go numbers option one signs mising easy togle option phone keyboard ap keyboard made post
349                                                 viber voice mesages chating viber best ap caling services even mesenger performs ap optimised newest version new option introduced alows se friends sen mesage group wel working makes litle bit mad finaly function introduced viber stil working
701                                                                                                                                      stil best mini like new lok news front fast eficient though overal best around browsers id go maxthon yandex opera mini gold though uc mini coming along neds
182                                                                                                                

In [482]:
df[df['polarity']==0]['review'].sample(10)

65              neds actual update twiter slows twets continue even takes long time load twet use moblie site version actualy use
166                                                      like review take advice cant save data though always find chest uninstal
308                          download pics used able download pics conversation save option long pres gives every option download
338    option separate viber se viber contacts separately shows together last update god cal drops video cal hear person properly
427                                     se change friend change nicknames mine setings change color anything fix please faster gr
36                             update uninstals everytime try update uninstals ap want know using data time update driving please
392                    major anoyance new update show online define online gogle using timestamps please ad dark mode feature wel
699                                                     cant download big things cant down

Save the model 


In [483]:
#filename = '../modelo_final.sav' #use absolute path
#pickle.dump(nb, open('../models/modelo_final.sav', 'wb'))
